# Module 6: Custom Exploratory Data Analysis
**Author: Caleb Sellinger**
**Date: 21 February 2025**
### Purpose:
This project will explore affordable rental housing in the City of Chicago. We will go through the processes, methods, and techniques used to transform and manipulate data, as well as an analysis of the dataset.

## Section 1: Fetching Data
First, we will import any libraries that we need.

In [37]:
import pathlib
import requests
import pandas as pd
from utils_logger import logger
import sqlite3
import plotly.express as px

Then, we will retrieve the data from the URL and write it into a file following the path "datasets" > "[filename].csv"

In [38]:
url = "https://data.cityofchicago.org/api/views/s6ha-ppgi/rows.csv?accessType=DOWNLOAD"
filename = "Affordable_Rental_Housing_Developments.csv"
folder = "datasets"

if not url:
    logger.error(
        "The URL provided is empty or does not exist. Please provide a valid URL."
    )

file_path = pathlib.Path(folder).joinpath(filename)

"""Retrieves data from URL"""
try:
    logger.info(f"Retrieving CSV file from {url}")
    response = requests.get(url)
    response.raise_for_status()
except requests.exceptions.HTTPError as http_err:
        logger.error(f"HTTP error: {http_err}")
except requests.exceptions.RequestException as req_err:
        logger.error(f"Request error: {req_err}")

"""Writes data from URL into file"""
try:
    logger.info(f"Writing data to file: {filename}")
    file_path.parent.mkdir(parents=True, exist_ok=True)
    file = file_path.open('w')
    file.write(response.text)
    file.close()
    logger.info(f"SUCCESS: data written to new file {filename}")
except IOError:
    logger.error(f"Error writing to file: {IOError}")

2025-02-20 22:41:34.576 | INFO     | __main__:<module>:14 - Retrieving CSV file from https://data.cityofchicago.org/api/views/s6ha-ppgi/rows.csv?accessType=DOWNLOAD
2025-02-20 22:41:35.098 | INFO     | __main__:<module>:24 - Writing data to file: Affordable_Rental_Housing_Developments.csv
2025-02-20 22:41:35.101 | INFO     | __main__:<module>:29 - SUCCESS: data written to new file Affordable_Rental_Housing_Developments.csv


We can quickly look at what the data by putting it into a dataframe and using the .info() method.

In [39]:
input_file = pathlib.Path(
    folder, filename
)

# was getting a utf-8 decode error, so I changed the encoding to latin-1
df = pd.read_csv(input_file, encoding='latin-1')

# print(df.head())
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 598 entries, 0 to 597
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Community Area Name    598 non-null    object 
 1   Community Area Number  598 non-null    int64  
 2   Property Type          598 non-null    object 
 3   Property Name          598 non-null    object 
 4   Address                598 non-null    object 
 5   Zip Code               598 non-null    int64  
 6   Phone Number           598 non-null    object 
 7   Management Company     598 non-null    object 
 8   Units                  597 non-null    float64
 9   X Coordinate           598 non-null    float64
 10  Y Coordinate           598 non-null    float64
 11  Latitude               598 non-null    float64
 12  Longitude              598 non-null    float64
 13  Location               589 non-null    object 
dtypes: float64(5), int64(2), object(7)
memory usage: 65.5+ KB


Now that we know what the data looks like, we can store it into our database.The following code creates a an SQL table to store our data and a database to put our table in.

In [40]:
# Define the database file in the current root project directory
db_file = pathlib.Path("db.sqlite3")

"""Creates a database. Connecting for the first time
will create a new database file if it doesn't exist yet."""
try:
    conn = sqlite3.connect(db_file)
    conn.close()
    print("Database created successfully.")
except sqlite3.Error as e:
    print("Error creating the database:", e)

"""Inserts the csv data into table."""
try:
    with sqlite3.connect(db_file) as conn:
        data_path = pathlib.Path("datasets", "Affordable_Rental_Housing_Developments.csv")
        df = pd.read_csv(data_path,encoding='latin-1')
        # use the pandas DataFrame to_sql() method to insert data
        # pass in the table name and the connection
        # if_exists drops table if exists
        df.to_sql("data", conn, if_exists="replace", index=False)
        print("Data inserted successfully.")
except (sqlite3.Error, pd.errors.EmptyDataError, FileNotFoundError) as e:
    print("Error inserting data:", e)

Database created successfully.
Data inserted successfully.


Here we can quickly see the top 5 rows of the table that was created running the code above.

In [41]:
pd.read_sql("SELECT * FROM data", con=sqlite3.connect(db_file)).head()

,Community Area Name,Community Area Number,Property Type,Property Name,Address,Zip Code,Phone Number,Management Company,Units,X Coordinate,Y Coordinate,Latitude,Longitude,Location
0,Avondale,21,Multifamily,Hairpin Lofts,3414 W. Diversey Ave.,60647,773-292-6360,Leasing & Management Co. Inc.,25.0,1153078.890,1918447.998,41.932073,-87.712872,None
1,Loop,32,ARO,1000M,1000 S. Michigan Ave.,60605,312-820-1000,Willow Bridge,23.0,1177375.505,1895971.036,41.869878,-87.624269,None
2,Logan Square,22,ARO,2556 Armtiage LLC,2556 W. Armitage Ave,60647,773-252-0600,North Clybourn Group,1.0,1158751.315,1913231.215,41.917643,-87.692170,"(41.917642826462, -87.6921699562562)"
3,Douglas,35,Multifamily,South Park Plaza,2600 S. King Dr.,60616,312-674-9210,Woodlawn Comm. Dev. Corp.,134.0,1179206.472,1887158.196,41.845653,-87.617816,"(41.8456529117633, -87.6178163910093)"
4,Near West Side,28,ARO,The Rosie,1461 S. Blue Island Ave.,60608,872-259-7452,The FLATS,7.0,1168331.384,1892984.019,41.861881,-87.657558,"(41.86188117554516, -87.65755843617394)"


A little table prettifying. For the NULL values in Location column, I wrote an sql script to update the NULL values to combine the Latitude and Longitude and formatted it appropriately.

In [42]:
try:
    with sqlite3.connect(db_file) as conn:
        sql_file = pathlib.Path("sql", "fill_nulls.sql")
        with open(sql_file, "r") as file:
            sql_script = file.read()
        conn.cursor().execute(sql_script)
except (sqlite3.Error, FileNotFoundError) as e:
    print("Error running sql:", e)

pd.read_sql("SELECT * FROM data", con=sqlite3.connect(db_file)).head()

,Community Area Name,Community Area Number,Property Type,Property Name,Address,Zip Code,Phone Number,Management Company,Units,X Coordinate,Y Coordinate,Latitude,Longitude,Location
0,Avondale,21,Multifamily,Hairpin Lofts,3414 W. Diversey Ave.,60647,773-292-6360,Leasing & Management Co. Inc.,25.0,1153078.890,1918447.998,41.932073,-87.712872,"(41.93207259, -87.71287204)"
1,Loop,32,ARO,1000M,1000 S. Michigan Ave.,60605,312-820-1000,Willow Bridge,23.0,1177375.505,1895971.036,41.869878,-87.624269,"(41.86987759, -87.6242687)"
2,Logan Square,22,ARO,2556 Armtiage LLC,2556 W. Armitage Ave,60647,773-252-0600,North Clybourn Group,1.0,1158751.315,1913231.215,41.917643,-87.692170,"(41.917642826462, -87.6921699562562)"
3,Douglas,35,Multifamily,South Park Plaza,2600 S. King Dr.,60616,312-674-9210,Woodlawn Comm. Dev. Corp.,134.0,1179206.472,1887158.196,41.845653,-87.617816,"(41.8456529117633, -87.6178163910093)"
4,Near West Side,28,ARO,The Rosie,1461 S. Blue Island Ave.,60608,872-259-7452,The FLATS,7.0,1168331.384,1892984.019,41.861881,-87.657558,"(41.86188117554516, -87.65755843617394)"


Here we can see that the top 3 most affordable housing types

In [43]:
df.value_counts("Property Type").head(3)

Property Type
Multifamily    259
ARO            156
Senior         106
Name: count, dtype: int64

The following code produces a scatter plot on a map. You will need to adjust the height according to your window size. If you hover over each marker, you can see a quick view of information. Currently displays address, property type, and number of units available. Plot layout reference here: https://plotly.com/python/reference/

In [54]:
# plotting map using plotly
fig = px.scatter_map(
    df,
    lat=df["Latitude"],
    lon=df["Longitude"],
    color=df["Property Type"],
    zoom=10,
    map_style="open-street-map",
    title="Location of Properties"
)

# settings not explicitly in scatter_map method for the map
fig.update_layout(
    # Can adjust accordingly
    height=800,
    # Width is automatically adjusted
    title_font_weight=1000,
    title_x=0.5,
    legend=dict(
        title_text="Property Types",
        title_font_weight=1000,
        bordercolor="Black",
        borderwidth=2,
        bgcolor="whitesmoke",
    ),
    map=dict(
        # North-South orientation
        bearing=0,
        # Center map on the middle point
        center=dict(lat=df.Latitude.mean(), lon=df.Longitude.mean()),
        pitch=25,
    ),
    autosize=True,
    paper_bgcolor="Lightsteelblue"
)
fig.update_traces(
    marker=dict(size=20),
    hovertemplate=(
        "Address: %{text}<br>"
        "Property Type: %{hovertext}<br>"
        "No. of Units: %{customdata}<br>"
        "<extra></extra>"  # Hide the default trace information (e.g., the trace name)
    ),
    text=df["Address"],
    customdata=df["Units"],
    hovertext=df["Property Type"]
)

fig.show()